# 各種法の統計量の table の生成

In [40]:
using IntervalArithmetic
using LaTeXStrings
using Plots

In [41]:
include("./heatmap/index.jl")
include("./method1/index.jl")
include("./method2/index.jl")
include("./method3/index.jl")
include("./method4/index.jl")
include("./intervalPCM/index.jl")
include("./twofoldInterval/index.jl")
include("./twofoldIntervalPCM/index.jl")

print()

In [42]:
function method1(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    A = intervalPCM2TwofoldIntervalPCM(A₁, A₂)
    result = solveLP_m1(A)
    Â = updatePCM_m1(A, result)
    return Â
end

method1 (generic function with 1 method)

In [43]:
function  method2(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    εᶿ = solveIndividualCancellingLP_m2([A₁, A₂])
    result = solveApproximationLP_m2([A₁, A₂], εᶿ)
    A = importance2TwofoldIntervalPCM_m2(result)
    return A
end

method2 (generic function with 1 method)

In [44]:
function method3(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    approximationResults = solveApproximationLP_m3.([A₁, A₂])
    tBoundaries = calculateTBoundaries_m3.(approximationResults)
    result = solveConcatLP_m3(approximationResults, tBoundaries)
    Â = generatePCM_m3(result)
    return Â
end

method3 (generic function with 1 method)

In [45]:
function method4(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    results = solveApproximationLP_m4.([A₁, A₂])
    A = generateTwofoldIntervalMatrix_m4(results)
    result = solveCancellingLP_m4(A)
    Â = updatePCM_m4(A, result)
    return Â
end

method4 (generic function with 1 method)

In [46]:
# 対角成分以外の算術平均
function simmean(A::Matrix{T})::T where {T <: Real}
    m, n = size(A)

    if m != n return 0 end
    if n == 1 return 0 end

    a = 0
    for i = 1:m, j = 1:n
        if i == j continue end
        a += A[i,j]
    end

    return a / n / (n-1)
end

# 対角成分以外の分散
function simvar(A::Matrix{T})::T where {T <: Real}
    m, n = size(A)

    if m != n return 0 end
    if n == 1 return 0 end

    μ = simmean(A)

    a = 0
    for i = 1:m, j = 1:n
        if i == j continue end
        a += (A[i,j])^2
    end
    a /= n * (n - 1)

    return a - μ^2
end

function plotAll(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    m = 2 # DM の数

    matrices = [A₁, A₂]
    resultMatrices = twofoldIntervalMatrix2intervalMatrices.(
        [method1(A₁, A₂), method2(A₁, A₂), method3(A₁, A₂), method4(A₁, A₂)])
    
    a = []; indmat = []
    for k = 1:length(resultMatrices)
        indrow = []
        Âₖ⁻, Âₖ⁺ = resultMatrices[k]
        for l = 1:m
            heatmap⁻, indices⁻ = plotConincidenceIndices(matrices[l], Âₖ⁻, L"~~")
            heatmap⁺, indices⁺ = plotConincidenceIndices(matrices[l], Âₖ⁺, L"~~")

            push!(indrow, indices⁻); push!(indrow, indices⁺)
        end

        push!(a, [Âₖ⁻, Âₖ⁺])
        push!(indmat, indrow)
    end

    tabμ⁻ = ""; tabV⁻ = ""
    tabμ⁺ = ""; tabV⁺ = ""
    tabρ = ""
    for i = eachindex(a)
        μ₁⁻ = simmean(indmat[i][1]); μ₁⁺ = simmean(indmat[i][2])
        μ₂⁻ = simmean(indmat[i][3]); μ₂⁺ = simmean(indmat[i][4])
        V₁⁻ = simvar(indmat[i][1]); V₁⁺ = simvar(indmat[i][2])
        V₂⁻ = simvar(indmat[i][3]); V₂⁺ = simvar(indmat[i][4])
        ρ⁻ = (μ₁⁻ + μ₂⁻) / 2; ρ⁺ = (μ₁⁺ + μ₂⁺) / 2

        tabμ⁻ *= "$(round(μ₁⁻, digits=3)) & $(round(μ₂⁻, digits=3)) \\\\\n"
        tabμ⁺ *= "$(round(μ₁⁺, digits=3)) & $(round(μ₂⁺, digits=3)) \\\\\n"
        tabV⁻ *= "$(round(V₁⁻, digits=4)) & $(round(V₂⁻, digits=4)) \\\\\n"
        tabV⁺ *= "$(round(V₁⁺, digits=4)) & $(round(V₂⁺, digits=4)) \\\\\n"
        tabρ *= "$(round(ρ⁻, digits=3)) & $(round(ρ⁺, digits=3)) \\\\\n"
    end

    println("tabμ⁻"); println(tabμ⁻); println()
    println("tabμ⁺"); println(tabμ⁺); println()
    println("tabV⁻"); println(tabV⁻); println()
    println("tabV⁺"); println(tabV⁺); println()
    println("tabρ"); println(tabρ)
end

plotAll (generic function with 2 methods)

# exmple 1

In [47]:
A₁ = [
    1..1 1..2 (2/3)..(4/3) 1..(4/3)
    (1/2)..1 1..1 (4/9)..1 (2/3)..1
    (3/4)..(3/2) 1..(9/4) 1..1 1..(3/2)
    (3/4)..1 1..(3/2) (2/3)..1 1..1
]
A₂ = A₁

display(L"A_1 = A_2 = %$(intervalMatrixLaTeXString(A₁))")

plotAll(A₂, A₂)

L"$A_1 = A_2 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 1.0, 2.0 \right] & \left[ 0.667, 1.333 \right] & \left[ 1.0, 1.333 \right] \\ \left[ 0.5, 1.0 \right] & \left[ 1.0, 1.0 \right] & \left[ 0.444, 1.0 \right] & \left[ 0.667, 1.0 \right] \\ \left[ 0.75, 1.5 \right] & \left[ 1.0, 2.25 \right] & \left[ 1.0, 1.0 \right] & \left[ 1.0, 1.5 \right] \\ \left[ 0.75, 1.0 \right] & \left[ 1.0, 1.5 \right] & \left[ 0.667, 1.0 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}$"

tabμ⁻
1.0 & 1.0 \\
1.0 & 1.0 \\
1.0 & 1.0 \\
1.0 & 1.0 \\


tabμ⁺
1.0 & 1.0 \\
1.0 & 1.0 \\
1.0 & 1.0 \\
1.0 & 1.0 \\


tabV⁻
0.0 & 0.0 \\
0.0 & 0.0 \\
0.0 & 0.0 \\
0.0 & 0.0 \\


tabV⁺
0.0 & 0.0 \\
0.0 & 0.0 \\
0.0 & 0.0 \\
0.0 & 0.0 \\


tabρ
1.0 & 1.0 \\
1.0 & 1.0 \\
1.0 & 1.0 \\
1.0 & 1.0 \\



# example 2

In [48]:
A₁ = [
    1..1 1..2 (2/3)..(4/3) 1..(4/3)
    (1/2)..1 1..1 (4/9)..1 (2/3)..1
    (3/4)..(3/2) 1..(9/4) 1..1 1..(3/2)
    (3/4)..1 1..(3/2) (2/3)..1 1..1
]
_, n = size(A₁)
A₂ = fill(1..1, (n, n))
for i = 1:n, j = 1:n
    if i == j continue end
    A₂[i,j] = (A₁[i,j].lo * 4/5)..(A₁[i,j].hi * 5/4)
end

display(L"A_1 = %$(intervalMatrixLaTeXString(A₁))")
display(L"A_2 = %$(intervalMatrixLaTeXString(A₂))")

plotAll(A₁, A₂)

L"$A_1 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 1.0, 2.0 \right] & \left[ 0.667, 1.333 \right] & \left[ 1.0, 1.333 \right] \\ \left[ 0.5, 1.0 \right] & \left[ 1.0, 1.0 \right] & \left[ 0.444, 1.0 \right] & \left[ 0.667, 1.0 \right] \\ \left[ 0.75, 1.5 \right] & \left[ 1.0, 2.25 \right] & \left[ 1.0, 1.0 \right] & \left[ 1.0, 1.5 \right] \\ \left[ 0.75, 1.0 \right] & \left[ 1.0, 1.5 \right] & \left[ 0.667, 1.0 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}$"

L"$A_2 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 0.8, 2.5 \right] & \left[ 0.533, 1.667 \right] & \left[ 0.8, 1.667 \right] \\ \left[ 0.4, 1.25 \right] & \left[ 1.0, 1.0 \right] & \left[ 0.356, 1.25 \right] & \left[ 0.533, 1.25 \right] \\ \left[ 0.6, 1.875 \right] & \left[ 0.8, 2.812 \right] & \left[ 1.0, 1.0 \right] & \left[ 0.8, 1.875 \right] \\ \left[ 0.6, 1.25 \right] & \left[ 0.8, 1.875 \right] & \left[ 0.533, 1.25 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}$"

tabμ⁻
1.0 & 0.519 \\
1.0 & 0.519 \\
0.993 & 0.522 \\
1.0 & 0.519 \\


tabμ⁺
0.519 & 1.0 \\
0.519 & 1.0 \\
0.52 & 0.997 \\
0.519 & 1.0 \\


tabV⁻
0.0 & 0.0073 \\
0.0 & 0.0073 \\
0.0001 & 0.007 \\
0.0 & 0.0073 \\


tabV⁺
0.0073 & 0.0 \\
0.0073 & 0.0 \\
0.0074 & 0.0 \\
0.0073 & 0.0 \\


tabρ
0.759 & 0.759 \\
0.759 & 0.759 \\
0.758 & 0.759 \\
0.759 & 0.759 \\



<!-- ともに弱い整合性を持ち， $A_{1ij} \subseteq A_{2ij}$

```
A₁ = [
    1..1 2..5 2..4 2..3
    (1/5)..(1/2) 1..1 (1/5)..4 (1/5)..3
    (1/4)..(1/2) (1/4)..5 1..1 (1/4)..3
    (1/3)..(1/2) (1/3)..5 (1/3)..4 1..1
]
A₂ = fill(1..1, size(A₁))
_, n = size(A₁)
for i = 1:n, j = 1:n
    if i == j continue end
    A₂[i,j] = (A₁[i,j].lo * 1/1.1)..(A₁[i,j].hi * 1.1)
end

display(L"A_1 = %$(intervalMatrixLaTeXString(A₁))")
display(L"A_2 = %$(intervalMatrixLaTeXString(A₂))")

display(isConsistentIntervalPCM(A₁))
display(isWeaklyConsistentIntervalPCM(A₁))
display(isConsistentIntervalPCM(A₂))
display(isWeaklyConsistentIntervalPCM(A₂))

plotAll(A₁, A₂)
``` -->

# example 3

推移率が成り立つ通常の $4 \times 4$ PCM に幅を与えて整合していない行列　2 つを作る

$\begin{bmatrix} 1/2 & 2 & 4 & 4 \\ 1/2 & 1 & 2 & 2 \\ 1/4 & 1/2 & 1 & 1 \\ 1/4 & 1/2 & 1 & 1 \end{bmatrix}$ に自然対数スケールで $[0, 0.03]$ の一様分布に従う乱数の分だけ上下に幅をもたせている

In [49]:
A = [
    1 2 4 4
    1/2 1 2 2
    1/4 1/2 1 1
    1/4 1/2 1 1
]

A₁ = randamizedIntervalPCM(A, 1234, 3e-2)
A₂ = randamizedIntervalPCM(A, 134, 3e-2)

display(L"A_1 = %$(intervalMatrixLaTeXString(A₁))")
display(L"A_2 = %$(intervalMatrixLaTeXString(A₂))")

plotAll(A₁, A₂)

L"$A_1 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 1.981, 2.033 \right] & \left[ 3.974, 4.109 \right] & \left[ 3.958, 4.048 \right] \\ \left[ 0.492, 0.505 \right] & \left[ 1.0, 1.0 \right] & \left[ 1.944, 2.048 \right] & \left[ 1.971, 2.045 \right] \\ \left[ 0.243, 0.252 \right] & \left[ 0.488, 0.515 \right] & \left[ 1.0, 1.0 \right] & \left[ 0.983, 1.022 \right] \\ \left[ 0.247, 0.253 \right] & \left[ 0.489, 0.507 \right] & \left[ 0.978, 1.017 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}$"

L"$A_2 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 1.998, 2.01 \right] & \left[ 3.974, 4.05 \right] & \left[ 3.962, 4.041 \right] \\ \left[ 0.497, 0.5 \right] & \left[ 1.0, 1.0 \right] & \left[ 1.945, 2.015 \right] & \left[ 1.942, 2.033 \right] \\ \left[ 0.247, 0.252 \right] & \left[ 0.496, 0.514 \right] & \left[ 1.0, 1.0 \right] & \left[ 0.972, 1.021 \right] \\ \left[ 0.247, 0.252 \right] & \left[ 0.492, 0.515 \right] & \left[ 0.979, 1.029 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}$"

tabμ⁻
0.668 & 0.879 \\
0.568 & 0.728 \\
0.682 & 0.749 \\
0.708 & 0.746 \\


tabμ⁺
0.846 & 0.66 \\
0.793 & 0.606 \\
0.819 & 0.628 \\
0.801 & 0.664 \\


tabV⁻
0.0569 & 0.018 \\
0.0671 & 0.0256 \\
0.024 & 0.0209 \\
0.0332 & 0.0262 \\


tabV⁺
0.0098 & 0.0639 \\
0.0109 & 0.0503 \\
0.0167 & 0.0356 \\
0.0251 & 0.046 \\


tabρ
0.773 & 0.753 \\
0.648 & 0.7 \\
0.715 & 0.723 \\
0.727 & 0.732 \\



# example 4

In [50]:
A = [
    1 2 3 10
    1/2 1 3 8
    1/3 1/3 1 3
    1/10 1/8 1/3 1
]

A₁ = randamizedIntervalPCM(A, 124, 4e-1)
A₂ = randamizedIntervalPCM(A, 411, 4e-1)

display(L"A_1 = %$(intervalMatrixLaTeXString(A₁))")
display(L"A_2 = %$(intervalMatrixLaTeXString(A₂))")

plotAll(A₁, A₂)

L"$A_1 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 1.415, 2.501 \right] & \left[ 2.252, 4.04 \right] & \left[ 8.141, 13.465 \right] \\ \left[ 0.4, 0.707 \right] & \left[ 1.0, 1.0 \right] & \left[ 2.122, 3.056 \right] & \left[ 6.481, 8.402 \right] \\ \left[ 0.248, 0.444 \right] & \left[ 0.327, 0.471 \right] & \left[ 1.0, 1.0 \right] & \left[ 2.174, 3.356 \right] \\ \left[ 0.074, 0.123 \right] & \left[ 0.119, 0.154 \right] & \left[ 0.298, 0.46 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}$"

L"$A_2 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 1.547, 2.232 \right] & \left[ 2.233, 3.723 \right] & \left[ 8.746, 12.652 \right] \\ \left[ 0.448, 0.646 \right] & \left[ 1.0, 1.0 \right] & \left[ 2.28, 3.644 \right] & \left[ 5.559, 10.701 \right] \\ \left[ 0.269, 0.448 \right] & \left[ 0.274, 0.439 \right] & \left[ 1.0, 1.0 \right] & \left[ 2.759, 3.774 \right] \\ \left[ 0.079, 0.114 \right] & \left[ 0.093, 0.18 \right] & \left[ 0.265, 0.362 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}$"

tabμ⁻
0.037 & 0.051 \\
0.294 & 0.264 \\
0.411 & 0.388 \\
0.488 & 0.405 \\


tabμ⁺
0.535 & 0.53 \\
0.53 & 0.481 \\
0.57 & 0.511 \\
0.59 & 0.544 \\


tabV⁻
0.0064 & 0.0121 \\
0.0698 & 0.0374 \\
0.0257 & 0.0524 \\
0.0462 & 0.0422 \\


tabV⁺
0.0355 & 0.0353 \\
0.0904 & 0.0327 \\
0.0187 & 0.0272 \\
0.0518 & 0.0596 \\


tabρ
0.044 & 0.533 \\
0.279 & 0.505 \\
0.399 & 0.54 \\
0.447 & 0.567 \\

